<a href="https://www.kaggle.com/code/tbillington/sql-querying-of-a-covid-19-dataset-bigquery?scriptVersionId=173243640" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

![](https://www.imperial.ac.uk/stories/covid-19-first-six-months/assets/E1BgLMkCC9/virus-2560-x-1440-2560x1440.jpeg)

# Querying a Covid-19 dataset using SQL in BigQuery

In [1]:
# BigQuery
from google.cloud import bigquery
bigquery_client = bigquery.Client(project='focused-waters-419608', location='US')

# Make an API request to fetch the dataset
dataset = bigquery_client.get_dataset('bigquery-public-data.covid19_open_data')

In [2]:
# Make an API request to fetch the dataset
dataset = bigquery_client.get_dataset('bigquery-public-data.covid19_open_data')

---

# Q1 - Calculate the average number of new confirmed cases per day for each country in the last month.

In [3]:
query1 = """
SELECT
    date,
    SUM(new_persons_fully_vaccinated) AS new_people_on_date_fully_vaccinated,
    country_name
FROM
    `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE date = '2021-03-01'
AND country_name = 'United Kingdom'
GROUP BY date, country_name;
"""

In [4]:
# Set up the query
query_job1 = bigquery_client.query(query1)

# Make an API request  to run the query and return a pandas DataFrame
housestyleAC = query_job1.to_dataframe()

# See the resulting table made from the query
print(housestyleAC)

         date  new_people_on_date_fully_vaccinated    country_name
0  2021-03-01                                66364  United Kingdom


-----

# Q2 - Calculate the average number of new confirmed cases per day for each country in the last month.

In [5]:
query2 = """
SELECT 
  country_name, 
  avg_new_confirmed,
  month,
  year 
FROM (
  SELECT
    country_name,
    EXTRACT(YEAR FROM date) AS year,
    EXTRACT(MONTH FROM date) AS month,
    AVG(new_confirmed) AS avg_new_confirmed
  FROM 
    `bigquery-public-data.covid19_open_data.covid19_open_data`
  GROUP BY 
    country_name, year, month
)
WHERE 
  month = 9 
  AND year = 2022
ORDER BY avg_new_confirmed DESC;
"""

query_job2 = bigquery_client.query(query2)
housestyleAC2 = query_job2.to_dataframe()
print(housestyleAC2.head())

  country_name  avg_new_confirmed  month  year
0  South Korea       65157.307692      9  2022
1      Germany       28621.846154      9  2022
2  Netherlands       17034.416373      9  2022
3       Taiwan        6562.992908      9  2022
4       Greece        5844.923077      9  2022


-----

# Q3 - Find the country with the highest percentage increase in confirmed cases compared to the previous day.

In [6]:
query3 = """
WITH ranked_data AS (
  SELECT
    country_name,
    date,
    new_confirmed_new,
    CONCAT(ROUND(((new_confirmed_new - LAG(new_confirmed_new, 1) OVER (PARTITION BY country_name ORDER BY date)) / LAG(new_confirmed_new, 1) OVER (PARTITION BY country_name ORDER BY date)) * 100, 2), '%') AS perc_inc,
    ROW_NUMBER() OVER (ORDER BY new_confirmed_new DESC) AS row_num
  FROM 
    (
    SELECT
      country_name,
      date,
      SUM(new_confirmed) AS new_confirmed_new
    FROM `bigquery-public-data.covid19_open_data.covid19_open_data` 
    WHERE new_confirmed > 0
    GROUP BY date, country_name
    )
)
SELECT * 
FROM ranked_data
WHERE row_num = 1
ORDER BY country_name, date DESC
LIMIT 100;
"""

query_job3 = bigquery_client.query(query3)
housestyleAC3 = query_job3.to_dataframe()
print(housestyleAC3.head())

  country_name        date  new_confirmed_new     perc_inc  row_num
0  Netherlands  2022-06-07           23239812  1635354.75%        1


---

# Q4 - Find the highest percentage increase in confirmed cases in each country

In [7]:
query4 = """SELECT
  country_name,
  date,
  new_confirmed_new,
  CONCAT(ROUND(((new_confirmed_new - LAG(new_confirmed_new, 1) OVER (PARTITION BY country_name ORDER BY date)) / LAG(new_confirmed_new, 1) OVER (PARTITION BY country_name ORDER BY date)) * 100, 2), '%') AS perc_inc
FROM 
  (
  SELECT
    country_name,
    date,
    SUM(new_confirmed) AS new_confirmed_new
  FROM `bigquery-public-data.covid19_open_data.covid19_open_data` 
  WHERE new_confirmed > 0
  GROUP BY date, country_name
  ) ranked_data
QUALIFY ROW_NUMBER() OVER (PARTITION BY country_name ORDER BY new_confirmed_new DESC) = 1
ORDER BY country_name, date DESC
LIMIT 1000;
"""

query_job4 = bigquery_client.query(query4)
housestyleAC4 = query_job4.to_dataframe()
print(housestyleAC4.head())

     country_name        date  new_confirmed_new perc_inc
0     Afghanistan  2021-08-31             107696  269140%
1         Albania  2022-01-17               2832  178.47%
2         Algeria  2022-01-24               2521   13.81%
3  American Samoa  2022-03-24                955  270.16%
4         Andorra  2022-01-24               1676   103.4%


----

# Q5 - Identify the top 10 countries with the highest ratio of deaths to confirmed cases.

In [8]:
query5 = """
SELECT
  country_name,
  SUM(new_deceased) AS total_deceased,
  SUM(new_confirmed) AS total_confirmed,
  ROUND(SUM(new_deceased) / SUM(new_confirmed) * 100, 2) || '%' AS ratio_deceased_to_confirmed
FROM
  `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE new_deceased > 0
AND new_confirmed > 0
GROUP BY country_name
ORDER BY ratio_deceased_to_confirmed DESC
LIMIT 10
"""

query_job5 = bigquery_client.query(query5)
housestyleAC5 = query_job5.to_dataframe()
print(housestyleAC5.head())

   country_name  total_deceased  total_confirmed ratio_deceased_to_confirmed
0         Sudan            4959            51619                       9.61%
1         Niger             309             3582                       8.63%
2       Liberia             293             3646                       8.04%
3        Gambia             367             5385                       6.82%
4  Sierra Leone             207             3157                       6.56%


-----

# Q6 - Calculate the total number of vaccinated individuals per day for each country in the last week.

In [9]:
query6 = """SELECT
  country_name,
  SUM(new_confirmed) AS total_confirmed,
  date
FROM
  `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE new_confirmed IS NOT Null
  AND date BETWEEN '2022-09-08' AND '2022-09-15'
GROUP BY country_name, date
ORDER BY country_name, date DESC
LIMIT
  1000
"""

query_job6 = bigquery_client.query(query6)
housestyleAC6 = query_job6.to_dataframe()
print(housestyleAC6.head())

  country_name  total_confirmed        date
0  Afghanistan              418  2022-09-15
1  Afghanistan             1018  2022-09-13
2  Afghanistan              222  2022-09-12
3  Afghanistan              170  2022-09-11
4  Afghanistan              365  2022-09-10


---

# Q7 - Determine the country with the highest number of cumulative vaccine doses administered per capita among countries with a population over 10 million.

In [10]:
query7 = """
SELECT
  population,
  country_name,
  MAX(cumulative_vaccine_doses_administered) AS max_vaccine_administered
FROM
  `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE population > 10000000
  AND cumulative_vaccine_doses_administered IS NOT Null
GROUP BY country_name, population
ORDER BY max_vaccine_administered DESC
LIMIT
  100
"""

query_job7 = bigquery_client.query(query7)
housestyleAC7 = query_job6.to_dataframe()
print(housestyleAC7.head())

  country_name  total_confirmed        date
0  Afghanistan              418  2022-09-15
1  Afghanistan             1018  2022-09-13
2  Afghanistan              222  2022-09-12
3  Afghanistan              170  2022-09-11
4  Afghanistan              365  2022-09-10
